# HW3

Submit via Slack. Due on Tuesday, April 13th, 2020, 6:29pm PST. You may work with one other person.

## TF-IDF

You are an analyst working at McDonalds as a store operations analyst, and charged with identifying areas for improvement for each franchise. Several metropolitan locations have been suffering recently from lower reviews.

Using the **mcdonalds-yelp-negative-reviews.csv** dataset, clean and parse the text reviews. Explain the decisions you make:
- why remove/keep stopwords?
- which stopwords to remove?
- stemming versus lemmatization?
- regex cleaning and substitution?
- adding in custom stopwords?
- what `n` for your `n-grams`?
- which words to collocate together?

Finally, generate a TF-IDF report that either **visualizes** or explains for a business (non-technical) stakeholder:
* the features your analysis showed that customers cited as reasons for a poor review
* the most common issues identified from your analysis that generated customer dissatisfaction.

Explain to what degree the TF-IDF findings make sense - what are its limitations?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from nltk.collocations import BigramCollocationFinder, BigramAssocMeasures
# import nltk
# nltk.download('wordnet')

In [2]:
mac_df = pd.read_csv("mcdonalds-yelp-negative-reviews.csv", encoding="latin1")

In [3]:
mac_df.head()

,_unit_id,city,review
0,679455653,Atlanta,"I'm not a huge mcds lover, but I've been to be..."
1,679455654,Atlanta,Terrible customer service. I came in at 9:30pm...
2,679455655,Atlanta,"First they ""lost"" my order, actually they gave..."
3,679455656,Atlanta,I see I'm not the only one giving 1 star. Only...
4,679455657,Atlanta,"Well, it's McDonald's, so you know what the fo..."


In [4]:
# Make all lowercase
mac_df["review"] = mac_df["review"].str.lower()

In [5]:
vectorizer = CountVectorizer(stop_words="english", binary=False)

X = vectorizer.fit_transform(mac_df["review"])

In [6]:
vec_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names()).T

vec_df["num_count"] = vec_df.sum(axis=1)

In [7]:
vec_df.sort_values("num_count", ascending=False)\
    .head(20)
#     .drop(list(set(stopwords.words('english'))), errors="ignore")

,0,1,2,3,4,5,6,7,8,9,...,1516,1517,1518,1519,1520,1521,1522,1523,1524,num_count
food,0,2,0,0,1,0,1,0,2,3,...,1,0,0,0,0,1,2,1,0,886
mcdonald,0,0,0,0,2,1,1,0,0,1,...,0,0,0,0,0,0,2,4,0,852
order,1,0,3,0,1,0,0,2,5,2,...,3,2,1,1,0,0,0,1,1,850
drive,1,0,0,0,0,0,0,1,3,0,...,0,2,0,0,0,0,0,0,0,693
just,0,1,1,0,0,0,0,0,0,1,...,0,2,0,1,0,0,1,3,2,596
mcdonalds,0,1,0,0,0,0,0,0,4,0,...,0,0,0,1,1,1,0,0,0,580
service,0,1,0,0,2,0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,539
time,1,0,0,0,1,0,0,1,1,2,...,0,1,0,0,0,1,0,1,0,532
like,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1,2,501
place,0,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,0,0,3,1,477


From the CountVectorizer, we can notice that many reviews contains "food", "order", "service", and "time"

## Regex Cleaning

In [8]:
# Hamburger Variation
mac_df["review"] = mac_df["review"].str.replace(r"\w*\s*burgers?", "burger")

In [9]:
# Big Macs
mac_df["review"] = mac_df["review"].str.replace(r"big\s*macs?", "burger")

Change types of burgers into burger to see how burgers served are reviewed

In [10]:
# McDonald's
mac_df["review"] = mac_df["review"].str.replace(r"(?:\bmcdonald(?:'?s?)?\b)|(?:\bmcds?\b)", "mcdonald")

Changed variations of McDonald's to mcdonald to add to stopwords later

In [11]:
# Punctuation Removal
mac_df["review"] = mac_df["review"].str.replace(r"[!|@|#|$|%|^|&|*|(|)|+|<|>|?|:|.|,|;|\"|\'|\\]", ' ')

In [12]:
# Whitespace
mac_df["review"] = mac_df["review"].str.replace(r"\s{2,}", ' ')

In [13]:
# Numbers
mac_df["review"] = mac_df["review"].str.replace(r"\d+\S*\d*\w*", "NUM_TOKEN")

Miscellaneous Regex

## Stemming

In [14]:
# stemmer = PorterStemmer()
stemmer = SnowballStemmer("english")

In [15]:
def stmmer_func(review):
    tokens = [stemmer.stem(token) for token in review.split()]
    return ' '.join(tokens)

Reason for choosing SnowballStemmer: Stemming is more appropriate than Lemmetization in our analysis because we want to search for keywords that causes the bad reviews. For example, we want to know if the order was the issue for the customer. And words "ordered" and "order" are likely to indicate that the review has something to do with order. Also, I chose SnowballStemmer over PorterStemmer because it is the improved version. 

## Customize Stopwords

In [16]:
stop_words = stopwords.words('english')

# Add stopword to stopwords list
stop_words.append("mcdonald")
stop_words = stop_words + [".",'.', ",",":", "''", "'s", "'", "``", "(", ")", "-", "!", "*", "?"]

# Remove from stopwords list
# stop_words.remove(["because", "most", "only"])

for w in ["because", "most", "only"]:
    stop_words.remove(w)

In [17]:
"because" in stop_words

False

I removed "mcdonald" as we already know the review is about mcdonald. \
I added because the word(s) after "because" signals what causes customers to leave a bad review. Same logic applies to "most" and "only". 

## Collocation

In [18]:
from nltk.corpus import wordnet

# https://gaurav5430.medium.com/using-nltk-for-lemmatizing-sentences-c1bfff963258
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return lemmatized_sentence

# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [19]:
lemmatizer = WordNetLemmatizer()

mac_df["tokenized_words"] = mac_df["review"].apply(lemmatize_sentence)

In [20]:
# Remove words in stopwords
mac_df["tokenized_words"] = mac_df["tokenized_words"].apply(lambda x: [word for word in x if word not in stop_words])

In [21]:
collocation_finder = BigramCollocationFinder.from_documents(mac_df["tokenized_words"])
measures = BigramAssocMeasures()

collocation_finder.nbest(measures.raw_freq, 15)

[('drive', 'thru'),
 ('NUM_TOKEN', 'NUM_TOKEN'),
 ('NUM_TOKEN', 'minute'),
 ('fast', 'food'),
 ('get', 'order'),
 ('customer', 'service'),
 ('NUM_TOKEN', 'star'),
 ('take', 'order'),
 ('go', 'back'),
 ('wait', 'NUM_TOKEN'),
 ('NUM_TOKEN', 'time'),
 ('order', 'wrong'),
 ('ice', 'cream'),
 ('order', 'NUM_TOKEN'),
 ('bad', 'ever')]

These are the words that tend to collocate together. 

## TF-IDF

In [22]:
vectorizer = TfidfVectorizer(input="mcdonalds-yelp-negative-reviews.csv",
                         encoding="latin1",
                         lowercase=True,
                         stop_words=stop_words,
                         ngram_range=(3,3))

In [23]:
corpus = list(mac_df["review"].values)

In [24]:
X = vectorizer.fit_transform(corpus)
terms = vectorizer.get_feature_names()
tf_idf = pd.DataFrame(X.toarray().transpose(), index=terms)

In [25]:
tf_idf = tf_idf.sum(axis=1)
score = pd.DataFrame(tf_idf, columns=["score"])
score.sort_values(by="score", ascending=False, inplace=True)

In [26]:
score.head(15)

,score
num_token num_token num_token,3.603224
get order right,3.298009
num_token minutes get,2.752656
waited num_token minutes,2.425034
went drive thru,2.403620
open num_token hours,2.209245
drive thru num_token,2.204614
drive thru window,2.191361
took num_token minutes,2.122771
got order wrong,2.108171


n-gram justification: I tried bigram for the analysis, but most words with high scores are usually collocated words like drive thru and ice cream. And quadgram simply has too many words that it does not appear in top rows. That is why I chose trigram. I acknowledge treating collocated words into one word will improve the analysis much better.

Most of the bad reviews are order related. Customers either did not get what they ordered or it took too long to get what they ordered, or even both. It seems like drive through and customer service need improvements as well.

It is consistent with the CountVectorizer analysis above as words like order, service were frequent words. However, none of tokens with high TF-IDF scores are food related as opposed to what we have observed in CounterVectorizer. There may be many reasons for that, but my assumption is that food is weighted down as they appear in many reviews. Also, there might be many variations around the word food, and thus not show up as much as the single word "food" does.  

## Product Attribution (Feature Engineering and Regex Practice)

Download the [dataset](https://dso-560-nlp-text-analytics.s3.amazonaws.com/truncated_catalog.csv) from the class S3 bucket (`dso560-nlp-text-analytics`).

In preparation for the group project, our client company has provided a dataset of women's clothing products they are considering cataloging. 

1. Filter for only **women's clothing items**.

2. For each clothing item:

* Identify its **category**:
```
Bottom
One Piece
Shoe
Handbag
Scarf
```
* Identify its **color**:
```
Beige
Black
Blue
Brown
Burgundy
Gold
Gray
Green
Multi 
Navy
Neutral
Orange
Pinks
Purple
Red
Silver
Teal
White
Yellow
```

Your output will be the same dataset, except with **3 additional fields**:
* `is_womens_clothing`
* `product_category`
* `colors`

`colors` should be a list of colors, since it is possible for a piece of clothing to have multiple colors.

In [27]:
import re

In [28]:
prod_df = pd.read_csv("truncated_catalog.csv")

In [29]:
cols = prod_df.columns.to_list()

for col in cols:
    prod_df[col] = prod_df[col].str.lower()

In [30]:
# Fill Null Values
prod_df.iloc[:, :7] = prod_df.iloc[:, :7].fillna('')

In [31]:
prod_df.head(3)

,brand,name,description,brand_category,brand_canonical_url,details,tsv
0,fila,original fitness sneakers,vintage fitness leather sneakers with logo pri...,themensstore/shoes/sneakers/lowtop,https://www.saksfifthavenue.com/fila-original-...,leather/synthetic upper\nlace-up closure\ntext...,"'design':12 'fila':1a 'fit':3a,6 'leather':7 '..."
1,chanel,hat,,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed & felt,'chanel':1a 'hat':2a
2,frame,petit oval buckle belt,a timeless leather belt crafted from smooth co...,accessories,https://frame-store.com/products/petit-oval-bu...,,"'belt':5a,9 'buckl':4a,21 'cowhid':13 'craft':..."


## is_womens_clothing

In [32]:
# Regex for capturing women related words 
woman_exp = "\bwi(?:fe|ves)|girls?|wom(?:a|e)n|lad(?:y|ies)|madams?|brides?|widows?|females?|femini\w*|maternal\w*|moms?\b"

# Search all columns
for col in cols:
    prod_df[f"is_womens_clothing_{col}"] = False
    
    # Find if women related words exist in the column
    prod_df[f"is_womens_clothing_{col}"] = prod_df[col].str.contains(woman_exp, case=False, flags=re.IGNORECASE, regex=True)
        
    print(f"{col} searched")
    
# If any of is_womens_clothing is True, then is_womens_clothing is True. Otherwise False
prod_df[f"is_womens_clothing"] = prod_df.iloc[:, 7:].any(axis=1)

# Drop other intermediate columns
col_to_drop = prod_df.iloc[0, 7:-1].index.to_list()
prod_df.drop(col_to_drop, axis=1, inplace=True)

brand searched
name searched
description searched
brand_category searched
brand_canonical_url searched
details searched
tsv searched


## product_category

In [33]:
# Expressions
bottom_exp = "(?:baggies|bottom|pant|jean|cord|chino|denim|legging|overall|short|trouser)(?:es|s)?"
one_piece_exp = "\bone[\S|\s]?piece|\w*dress|all[\S|\s]?in[\S|\s]?one\b"
shoe_exp = "(?:shoe|boot|cleat|hopper|trainer|flat|flip[\S|\s]?flop|heel|pump|slide|slipper|skate|sneaker|wedge)(?:s|es)?"
handbag_exp = "(?:\w* ?bags?|clutch(?:es)?|satchels?)"
scarf_exp = "(?:\w* ?scar(?:f|(?:ves))?|snoods?|stoles?|boas?|sarongs?)"

cats_list = ["Bottom", "One_Piece", "Shoe", "Handbag", "Scarf"]
exps_list = [bottom_exp, one_piece_exp, shoe_exp, handbag_exp, scarf_exp]

# For each product category
for cat, exp in zip(cats_list, exps_list):
    prod_df.loc[:,cat] = 0
    
    for col in cols:
        # Add the number of occurrences in all columns
        prod_df[cat] = prod_df[col].str.findall(exp, flags=re.IGNORECASE).apply(lambda x: len(x))
        print(f"{cat} in {col} searched")

Bottom in brand searched
Bottom in name searched
Bottom in description searched
Bottom in brand_category searched
Bottom in brand_canonical_url searched
Bottom in details searched
Bottom in tsv searched
One_Piece in brand searched
One_Piece in name searched
One_Piece in description searched
One_Piece in brand_category searched
One_Piece in brand_canonical_url searched
One_Piece in details searched
One_Piece in tsv searched
Shoe in brand searched
Shoe in name searched
Shoe in description searched
Shoe in brand_category searched
Shoe in brand_canonical_url searched
Shoe in details searched
Shoe in tsv searched
Handbag in brand searched
Handbag in name searched
Handbag in description searched
Handbag in brand_category searched
Handbag in brand_canonical_url searched
Handbag in details searched
Handbag in tsv searched
Scarf in brand searched
Scarf in name searched
Scarf in description searched
Scarf in brand_category searched
Scarf in brand_canonical_url searched
Scarf in details searched


In [34]:
# Finds the category that has the highest score
prod_df["product_category"] = prod_df.iloc[:, 8:].apply(lambda x: x.idxmax() if x.sum() != 0 else None, axis=1)

Acknowledgement: idxmax fails to identify the category when there is a tie. idxmax fails to break the tie as it chooses the index of former tie. For example, if both shoe and handbag show up once in a product, it idxmax will choose shoe instead of handbag, which may not be true.

In [35]:
prod_df.drop(["Bottom", "One_Piece", "Shoe", "Handbag", "Scarf"], axis=1, inplace=True)

In [36]:
prod_df.head(3)

,brand,name,description,brand_category,brand_canonical_url,details,tsv,is_womens_clothing,product_category
0,fila,original fitness sneakers,vintage fitness leather sneakers with logo pri...,themensstore/shoes/sneakers/lowtop,https://www.saksfifthavenue.com/fila-original-...,leather/synthetic upper\nlace-up closure\ntext...,"'design':12 'fila':1a 'fit':3a,6 'leather':7 '...",False,Shoe
1,chanel,hat,,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed & felt,'chanel':1a 'hat':2a,False,None
2,frame,petit oval buckle belt,a timeless leather belt crafted from smooth co...,accessories,https://frame-store.com/products/petit-oval-bu...,,"'belt':5a,9 'buckl':4a,21 'cowhid':13 'craft':...",False,None


## colors

In [37]:
color_exp = "(?:Beige|Black|Blue|Brown|Burgund(?:y|ies)|Gold|Gra(?:y|ies)|Green|Multi|Nav(?:y|ies)|Neutral|Orange|Pink|Purple|Red|Silver|Teal|White|Yellow)s?"

for col in cols:
    prod_df[f"colors_{col}"] = None
    
    # Find colors
    prod_df[f"colors_{col}"] = prod_df[col].str.findall(color_exp, flags=re.IGNORECASE).apply(lambda x: ''.join(x))
        
    print(f"{col} searched")

brand searched
name searched
description searched
brand_category searched
brand_canonical_url searched
details searched
tsv searched


In [38]:
# Append colors to the list
prod_df["colors"] = prod_df.iloc[:, 9:].apply(lambda x: list(set([color for color in x if color != ''])), axis=1)

In [39]:
# Drop columns
prod_df.drop(["colors_brand", "colors_name", "colors_description", "colors_brand_category",
              "colors_brand_canonical_url", "colors_details", "colors_tsv"],
             axis=1, inplace=True)

In [40]:
# Change color to "Multi" if multiple colors in a product
prod_df["colors"] = prod_df["colors"].apply(lambda x: x if len(x) < 2 else ["Multi"])
prod_df["colors"] = prod_df["colors"].apply(lambda x: None if len(x) == 0 else x)

In [41]:
prod_df

,brand,name,description,brand_category,brand_canonical_url,details,tsv,is_womens_clothing,product_category,colors
0,fila,original fitness sneakers,vintage fitness leather sneakers with logo pri...,themensstore/shoes/sneakers/lowtop,https://www.saksfifthavenue.com/fila-original-...,leather/synthetic upper\nlace-up closure\ntext...,"'design':12 'fila':1a 'fit':3a,6 'leather':7 '...",False,Shoe,None
1,chanel,hat,,unknown,https://www.saksfifthavenue.com/chanel-hat/pro...,wool tweed & felt,'chanel':1a 'hat':2a,False,None,None
2,frame,petit oval buckle belt,a timeless leather belt crafted from smooth co...,accessories,https://frame-store.com/products/petit-oval-bu...,,"'belt':5a,9 'buckl':4a,21 'cowhid':13 'craft':...",False,None,[Multi]
3,lilly pulitzer kids,little gir's & girl's ariana one-piece upf 50+...,pretty ruffle sleeves and trim elevate essenti...,"justkids/girls214/girls/swimwearcoverups,justk...",https://www.saksfifthavenue.com/lilly-pulitzer...,scoopneck\nadjustable straps\nflutter sleeves\...,'50':14a 'allov':28 'ariana':9a 'color':27 'el...,True,None,None
4,kissy kissy,baby girl's endearing elephants pima cotton co...,versatile convertible gown with elephant applique,justkids/baby024months/infantgirls/footiesrompers,https://www.saksfifthavenue.com/kissy-kissy-ba...,v-neckline\nlong sleeves\nfront snap closure\n...,"'appliqu':17 'babi':3a 'convert':10a,13 'cotto...",True,None,None
...,...,...,...,...,...,...,...,...,...,...
42368,mara hoffman,atlas oversized belted mélange wool coat,mélange beige and cream wool button fastenings...,clothing / coats / long,https://www.net-a-porter.com/us/en/product/117...,"fits true to size, take your normal size \ndes...",'100':21 'atlas':3a 'beig':10 'belt':5a 'breas...,False,None,[beige]
42369,philosophy di lorenzo serafini,cropped crochet-trimmed georgette top,"cream georgette ties at neck, concealed hook f...",clothing / tops / blouses,https://www.net-a-porter.com/us/en/product/111...,"fits true to size, take your normal size \nint...",'100':21 'back':20 'conceal':16 'cream':11 'cr...,False,None,None
42370,vanessa bruno,juna cotton-corduroy mini skirt,sand cotton-corduroy concealed hook and zip fa...,clothing / skirts / mini,https://www.net-a-porter.com/us/en/product/116...,"fits true to size, take your normal size \ntho...",'100':20 '35':25 '65':23 'acet':24 'back':19 '...,False,Bottom,None
42371,eve denim,annabel rigid mid-rise skinny jean,although mom jeans and boyfriend jeans are all...,women:clothing:jeans,https://pink.modaoperandi.com/eve-denim-r20/an...,button and zip fastening \ncomposition: 98% co...,"'add':36 'although':10 'annabel':3a,40 'boyfri...",True,Bottom,[pink]
